In [8]:
import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import datetime
from torch.utils.data import Dataset, DataLoader
import torch
from distutils.util import strtobool
import random

from scipy.stats import norm
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm
import pandas as pd
import importlib

# This is just until temporary implementation
import os 
import sys
cwd = os.getcwd()
sys.path.insert(0,cwd + '/../timetransformers')

import Transformer 
from data_handling import TimeSeriesDataset, download_data
from utils import convert_df_to_numpy, convert_tsf_to_dataframe

%matplotlib inline
%config InlineBackend.figure_format='retina'

In [3]:
device = torch.device("cpu")

importlib.reload(Transformer)

output_dim = 2  # To begin with we can use a Gaussian with mean and variance
d_model = 32
num_heads = 1
num_layers = 2
d_ff = 128
dropout = 0.0
num_distribution_layers = 4

max_seq_length = 10

transformer = Transformer.Decoder_Transformer(
    output_dim,
    d_model,
    num_heads,
    num_layers,
    d_ff,
    max_seq_length,
    dropout,
    num_distribution_layers,
    device=device,
).to(device)

In [7]:
data = torch.randn(2, 10)
custom_mask = torch.tensor(
    [
        [False, False, True, False, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
    ]
)
print(transformer.generate_mask(data, custom_mask=custom_mask))
# a = transformer.generate_mask(data, custom_mask=custom_mask)

tensor([[[[False, False, False, False, False, False, False, False, False, False],
          [False, False, False, False, False, False, False, False, False, False],
          [False, False,  True, False, False, False, False, False, False, False],
          [False, False,  True, False, False, False, False, False, False, False],
          [False, False,  True, False,  True, False, False, False, False, False],
          [False, False,  True, False,  True,  True, False, False, False, False],
          [False, False,  True, False,  True,  True,  True, False, False, False],
          [False, False,  True, False,  True,  True,  True,  True, False, False],
          [False, False,  True, False,  True,  True,  True,  True,  True, False],
          [False, False,  True, False,  True,  True,  True,  True,  True,  True]]],


        [[[ True, False, False, False, False, False, False, False, False, False],
          [ True,  True, False, False, False, False, False, False, False, False],
          [ 

In [24]:
datasets_to_load = {
    "electricity_hourly_dataset": "10.5281/zenodo.4656140",
}
dfs = download_data(datasets_to_load)

electricity_hourly_dataset.tsf already exists. Skipping download.


In [32]:
import pandas as pd

arr = np.array([1, 2, np.nan, 4, 5, 6, 7, 8, 9, 10])

for j in range(len(dfs[0]["series_value"])):
    dfs[0]["series_value"].iloc[j] = pd.Series(arr)

print(dfs[0]["series_value"][0])


# (
#     training_data_list,
#     test_data_list,
#     train_masks,
#     test_masks,
# ) = convert_df_to_numpy(dfs, 0.95)

0     1.0
1     2.0
2     NaN
3     4.0
4     5.0
5     6.0
6     7.0
7     8.0
8     9.0
9    10.0
dtype: float64


/var/folders/jd/x5xb5z5s2fq2qzq_69vsp0pw0000gn/T/ipykernel_1680/3561606090.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs[0]["series_value"].iloc[j] = pd.Series(arr)
/var/folders/jd/x5xb5z5s2fq2qzq_69vsp0pw0000gn/T/ipykernel_1680/3561606090.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs[0]["series_value"].iloc[j] = pd.Series(arr)
/var/folders/jd/x5xb5z5s2fq2qzq_69vsp0pw0000gn/T/ipykernel_1680/3561606090.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

In [33]:
(
    training_data_list,
    test_data_list,
    train_masks,
    test_masks,
) = convert_df_to_numpy(dfs, 0.95)

In [37]:
print(training_data_list)
print(train_masks)

[array([1., 2., 0., 4., 5., 6., 7., 8., 9.]), array([1., 2., 0., 4., 5., 6., 7., 8., 9.]), array([1., 2., 0., 4., 5., 6., 7., 8., 9.]), array([1., 2., 0., 4., 5., 6., 7., 8., 9.]), array([1., 2., 0., 4., 5., 6., 7., 8., 9.]), array([1., 2., 0., 4., 5., 6., 7., 8., 9.]), array([1., 2., 0., 4., 5., 6., 7., 8., 9.]), array([1., 2., 0., 4., 5., 6., 7., 8., 9.]), array([1., 2., 0., 4., 5., 6., 7., 8., 9.]), array([1., 2., 0., 4., 5., 6., 7., 8., 9.]), array([1., 2., 0., 4., 5., 6., 7., 8., 9.]), array([1., 2., 0., 4., 5., 6., 7., 8., 9.]), array([1., 2., 0., 4., 5., 6., 7., 8., 9.]), array([1., 2., 0., 4., 5., 6., 7., 8., 9.]), array([1., 2., 0., 4., 5., 6., 7., 8., 9.]), array([1., 2., 0., 4., 5., 6., 7., 8., 9.]), array([1., 2., 0., 4., 5., 6., 7., 8., 9.]), array([1., 2., 0., 4., 5., 6., 7., 8., 9.]), array([1., 2., 0., 4., 5., 6., 7., 8., 9.]), array([1., 2., 0., 4., 5., 6., 7., 8., 9.]), array([1., 2., 0., 4., 5., 6., 7., 8., 9.]), array([1., 2., 0., 4., 5., 6., 7., 8., 9.]), array([1.